# Import libraries

In [1]:
import numpy as np
import pandas as pd
import scipy.io
from sklearn.preprocessing import MinMaxScaler
import yaml

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import TensorDataset, DataLoader

In [3]:
from loss_functions import RMSELoss
from mat_utils import MatUtils
from transformer_model import TransformerModel

In [4]:
from dataset_utils import create_dataset
from evaluate_model import evaluate_model
from performance_metrics import r2, mae
from train import train_model

In [5]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA is available. Using GPU.")
else:
    device = torch.device('cpu')
    print("CUDA is not available. Using CPU.")

CUDA is not available. Using CPU.


# Define info from yaml file

In [6]:
with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)

In [7]:
batch_size = config["model"]["batch_size"]
d_model = config["model"]["d_model"]
epochs = config["model"]["epochs"]
input_dim = config["model"]["input_dim"]
output_dim = config["model"]["output_dim"]
num_heads = config["model"]["num_heads"]
num_layers = config["model"]["num_layers"]
d_ff = config["model"]["d_ff"]
dropout = config["model"]["dropout"]

In [8]:
learning_rate = config["training"]["learning_rate"]
optimizer = config["training"]["optimizer"]
save_model_path = config["training"]["save_model_path"]

In [9]:
train_filepath = config["data"]["file_paths"]["train"]
val_filepath = config["data"]["file_paths"]["validate"]
test_filepath = config["data"]["file_paths"]["test"]

In [10]:
print(train_filepath)
print(val_filepath)
print(test_filepath)

/Users/marilyn/Library/Mobile Documents/com~apple~CloudDocs/Documents/doctorate/ssdl/gps_bias/src/data/train/train_small.mat
/Users/marilyn/Library/Mobile Documents/com~apple~CloudDocs/Documents/doctorate/ssdl/gps_bias/src/data/validate/val_small.mat
/Users/marilyn/Library/Mobile Documents/com~apple~CloudDocs/Documents/doctorate/ssdl/gps_bias/src/data/test/test_small.mat


# Understand data

In [11]:
train_data = scipy.io.loadmat(train_filepath)
val_data = scipy.io.loadmat(val_filepath)
test_data = scipy.io.loadmat(test_filepath)

In [12]:
train_utils_instance = MatUtils(train_filepath)
validate_utils_instance = MatUtils(val_filepath)
test_utils_instance = MatUtils(test_filepath)

In [13]:
train_utils_instance.print_mat_file_content()
validate_utils_instance.print_mat_file_content()
test_utils_instance.print_mat_file_content()

Contents of /Users/marilyn/Library/Mobile Documents/com~apple~CloudDocs/Documents/doctorate/ssdl/gps_bias/src/data/train/train_small.mat:
Variable name: None, Shape: (1,)
Variable name: processed_final_clock_bias, Shape: (1, 304416)
Variable name: processed_broadcast_clock_bias, Shape: (1, 304416)
Variable name: processed_correction_value, Shape: (1, 304416)
Variable name: original_correction_value, Shape: (1, 304416)
Contents of /Users/marilyn/Library/Mobile Documents/com~apple~CloudDocs/Documents/doctorate/ssdl/gps_bias/src/data/validate/val_small.mat:
Variable name: None, Shape: (1,)
Variable name: processed_final_clock_bias, Shape: (1, 43488)
Variable name: processed_broadcast_clock_bias, Shape: (1, 43488)
Variable name: processed_correction_value, Shape: (1, 43488)
Variable name: original_correction_value, Shape: (1, 43488)
Contents of /Users/marilyn/Library/Mobile Documents/com~apple~CloudDocs/Documents/doctorate/ssdl/gps_bias/src/data/test/test_small.mat:
Variable name: original

In [14]:
train_broadcast_clock_bias = train_data.get('processed_broadcast_clock_bias')[0]
train_correction_value = train_data.get('processed_correction_value')[0]
val_broadcast_clock_bias = val_data.get('processed_broadcast_clock_bias')[0]
val_correction_value = val_data.get('processed_correction_value')[0]
test_broadcast_clock_bias = test_data.get('broadcast_clock_bias')[0]
test_correction_value = test_data.get('correction_value')[0]

In [15]:
print(train_broadcast_clock_bias.shape)
print(train_correction_value.shape)
print(val_broadcast_clock_bias.shape)
print(val_correction_value.shape)
print(test_broadcast_clock_bias.shape)
print(test_correction_value.shape)

(304416,)
(304416,)
(43488,)
(43488,)
(86976,)
(86976,)


In [16]:
combined_broadcast_clock_bias = np.concatenate((
    train_broadcast_clock_bias,
    val_broadcast_clock_bias,
    test_broadcast_clock_bias
))

In [17]:
combined_correction_value = np.concatenate((
    train_correction_value,
    val_correction_value,
    test_correction_value
))

In [18]:
print(combined_broadcast_clock_bias.shape)
print(combined_correction_value.shape)

(434880,)
(434880,)


In [19]:
broadcast_clock_bias_scaler = MinMaxScaler()
correction_value_scaler = MinMaxScaler()

In [20]:
broadcast_clock_bias_scaler.fit(combined_broadcast_clock_bias.reshape(-1,1)) # expects 2D array 
correction_value_scaler.fit(combined_correction_value.reshape(-1,1)) # expects 2D array 

MinMaxScaler()

In [21]:
train_broadcast_clock_bias_scaled = broadcast_clock_bias_scaler.transform(train_broadcast_clock_bias.reshape(-1, 1))
val_broadcast_clock_bias_scaled = broadcast_clock_bias_scaler.transform(val_broadcast_clock_bias.reshape(-1, 1))
test_broadcast_clock_bias_scaled = broadcast_clock_bias_scaler.transform(test_broadcast_clock_bias.reshape(-1, 1))

In [22]:
train_correction_value_scaled = correction_value_scaler.transform(train_correction_value.reshape(-1, 1))
val_correction_value_scaled = correction_value_scaler.transform(val_correction_value.reshape(-1, 1))
test_correction_value_scaled = correction_value_scaler.transform(test_correction_value.reshape(-1, 1))

In [23]:
train_broadcast_clock_bias_scaled = train_broadcast_clock_bias_scaled.squeeze()
val_broadcast_clock_bias_scaled = val_broadcast_clock_bias_scaled.squeeze()
test_broadcast_clock_bias_scaled = test_broadcast_clock_bias_scaled.squeeze()

train_correction_value_scaled = train_correction_value_scaled.squeeze()
val_correction_value_scaled = val_correction_value_scaled.squeeze()
test_correction_value_scaled = test_correction_value_scaled.squeeze()

In [24]:
print(train_broadcast_clock_bias_scaled.shape)
print(val_broadcast_clock_bias_scaled.shape)
print(test_broadcast_clock_bias_scaled.shape)

print(train_correction_value_scaled.shape)
print(val_correction_value_scaled.shape)
print(test_correction_value_scaled.shape)

(304416,)
(43488,)
(86976,)
(304416,)
(43488,)
(86976,)


In [25]:
train = pd.DataFrame({
    'broadcast_clock_bias': train_broadcast_clock_bias_scaled,
    'correction_value': train_correction_value_scaled
})

In [26]:
validate = pd.DataFrame({
    'broadcast_clock_bias': val_broadcast_clock_bias_scaled,
    'correction_value': val_correction_value_scaled
})

In [27]:
test = pd.DataFrame({
    'broadcast_clock_bias': test_broadcast_clock_bias_scaled,
    'correction_value': test_correction_value_scaled
})

In [28]:
def sliding_window(df: pd.DataFrame, sequence_length: int): 
    windows = [df.iloc[i : i + sequence_length].values for i in range(len(df) - sequence_length)]
    X = np.array(windows)
    return X

In [29]:
seq_len= 360  # 240 past + 120 future

In [30]:
X_train = sliding_window(df=train, sequence_length=seq_len)
X_train.shape

(304056, 360, 2)

In [31]:
X_val = sliding_window(df=validate, sequence_length=seq_len)
X_val.shape

(43128, 360, 2)

In [32]:
X_test = sliding_window(df=test, sequence_length=seq_len)
X_test.shape

(86616, 360, 2)

# Implement Model

In [33]:
model = TransformerModel(input_dim = 1, output_dim = 1, d_model = d_model)

In [34]:
# model = nn.Transformer(d_model=d_model, batch_first=True)

In [35]:
X_train = torch.tensor(X_train)
X_val = torch.tensor(X_val)
X_test = torch.tensor(X_test)

In [36]:
src_train = X_train[:, :240, 0].unsqueeze(-1)  # First 240 time steps as source
src_val = X_val[:, :240, 0].unsqueeze(-1)  # First 240 time steps as source
src_test = X_test[:, :240, 0].unsqueeze(-1)  # First 240 time steps as source

In [37]:
print(src_train.shape)
print(src_val.shape)
print(src_test.shape)

torch.Size([304056, 240, 1])
torch.Size([43128, 240, 1])
torch.Size([86616, 240, 1])


In [38]:
tgt_train = X_train[:, 240:, 1].unsqueeze(-1)  # Last 120 time steps as target (only correction_value)
tgt_val = X_val[:, 240:, 1].unsqueeze(-1)  # Last 120 time steps as target (only correction_value)
tgt_test = X_test[:, 240:, 1].unsqueeze(-1)  # Last 120 time steps as target (only correction_value)

In [39]:
print(tgt_train.shape)
print(tgt_val.shape)
print(tgt_test.shape)

torch.Size([304056, 120, 1])
torch.Size([43128, 120, 1])
torch.Size([86616, 120, 1])


In [40]:
src_train = src_train.float()
src_val = src_val.float()
src_test = src_test.float()

tgt_train = tgt_train.float()
tgt_val = tgt_val.float()
tgt_test = tgt_test.float()

In [41]:
train_dataset = TensorDataset(src_train, tgt_train)
val_dataset = TensorDataset(src_val, tgt_val)
test_dataset = TensorDataset(src_test, tgt_test)

In [42]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


In [43]:
loss_fn = RMSELoss()

In [44]:
import torch.optim as optim

# Define the optimizer (Adam in this case)
optimizer = optim.Adam(model.parameters(), lr=1e-4)  # Example learning rate


In [ ]:
train_losses, train_mae, val_losses, val_mae = train_model(model, d_model, train_dataloader, val_dataloader, loss_fn, optimizer, device, epochs)